In [1]:
# standard libraries
import math
import os
import tempfile

# standard scientific libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray, save, load
import pandas as pd
import seaborn as sns

# scikit-learn
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# tensorflow
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D, Conv2D, MaxPooling2D, Dropout, Flatten, Input, MaxPooling1D
from keras.optimizers import RMSprop

2024-04-29 13:53:03.864524: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
# TODO
# fragment length (8- 14)
# classifier vs. single-sequence (GNRA)
# with/without decoys

In [2]:
# Config (hyperparameters, figsize, metrics, etc.)

EPOCHS = 20
BATCH_SIZE = 16
mpl.rcParams['figure.figsize'] = (15, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      # TODO add F1, MCC
]

2024-04-29 13:53:09.600682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13762 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:30:00.0, compute capability: 7.5


In [3]:
# Plotting functions

def plot_loss(history, label, n):
  # Use a log scale on y-axis to show the wide range of values.
  plt.semilogy(history.epoch, history.history['loss'],
               color=colors[n], label='Train ' + label)
  # plt.semilogy(history.epoch, history.history['val_loss'],
  #              color=colors[n], label='Val ' + label,
  #              linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')

def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix')
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Non-GNRA Detected (True Negatives): ', cm[0][0])
  print('Non-GNRA Incorrectly Detected (False Positives): ', cm[0][1])
  print('GNRA Missed (False Negatives): ', cm[1][0])
  print('GNRA Detected (True Positives): ', cm[1][1])
  print('Total GNRA: ', np.sum(cm[1]))

def plot_metrics(history1, history2):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history1.epoch, history1.history[metric], color=colors[0], label='Conf1_Train')
    plt.plot(history1.epoch, history1.history['val_' + metric],
             color=colors[0], linestyle="--", label='Conf1_Test')
    plt.plot(history2.epoch, history2.history[metric], color=colors[1], label='Conf2_Train')
    plt.plot(history2.epoch, history2.history['val_' + metric],
             color=colors[1], linestyle="--", label='Conf2_Test')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

def plot_roc(name, labels, predictions, **kwargs):
  fpr, tpr, _ = sklearn.metrics.roc_curve(labels, predictions)  # fpr=false positive rate = fp/(fp+tn), tpr=true positive rate = tp/(tp+fn)

  plt.rcParams['font.size'] = '16'
  plt.plot(100*fpr, 100*tpr, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives rate = fp/(fp+tn)')
  plt.ylabel('True positives rate = tp/(tp+fn)')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

def plot_prc(name, labels, predictions, **kwargs):  #precision = tp / (tp + fp), recall = tp / (tp + fn)
    precision, recall, _ = sklearn.metrics.precision_recall_curve(labels, predictions)

    plt.rcParams['font.size'] = '9'
    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall = TP / (TP + FP)')
    plt.ylabel('Precision = TP / (TP + FN)')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [4]:
# Load dataset
dict_train = np.load('noAnno_train_14_nucleotide_array_with_cluster0.npz')
x_train = np.stack(dict_train['arr_0'], axis=0)
y_flt_train = np.load('noAnno_train_14_nucleotide_labels_with_cluster0.npy')
y_train = y_flt_train.astype(int)
print(y_train)

dict_test = np.load('noAnno_test_14_nucleotide_array_with_cluster0.npz')
x_test = np.stack(dict_test['arr_0'], axis=0)
y_flt_test = np.load('noAnno_test_14_nucleotide_labels_with_cluster0.npy')
y_test = y_flt_test.astype(int)
print(y_test)

print('Training features shape:', x_train.shape)
print('Test features shape:', x_test.shape)
print('Training labels shape:', y_train.shape)
print('Test labels shape:', y_test.shape)

[0 0 0 ... 0 0 0]
[1 0 0 ... 1 0 0]
Training features shape: (52739, 14, 4)
Test features shape: (26370, 14, 4)
Training labels shape: (52739,)
Test labels shape: (26370,)


In [5]:
neg, pos = np.bincount(y_train)+np.bincount(y_test)
total = neg + pos

print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

print('Training labels shape:', y_train.shape)
print('Test labels shape:', y_test.shape)

print('Training features shape:', x_train.shape)
print('Test features shape:', x_test.shape)

Examples:
    Total: 79109
    Positive: 5578 (7.05% of total)

Training labels shape: (52739,)
Test labels shape: (26370,)
Training features shape: (52739, 14, 4)
Test features shape: (26370, 14, 4)


In [6]:
def make_model1(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    model = Sequential([
        # The input shape is 14x4
        # This is the first convolution
        Conv1D(16, 3, strides=1, activation='relu', padding='same',
               input_shape=(14, 4),
               kernel_initializer='he_normal',
               bias_initializer='zeros'),
        MaxPooling1D(pool_size=2, strides=2),
        Dropout(0.2),
        Conv1D(32, 3, strides=1, activation='relu', padding='same',
               kernel_initializer='he_normal',
               bias_initializer='zeros'),
        MaxPooling1D(pool_size=2, strides=2),
        Dropout(0.2),
        Flatten(),
        #  neuron hidden layer
        Dense(224, activation='relu', bias_initializer=output_bias),  # 14/2(maxpooling)=7, 32*7 = 224
        Dropout(0.2),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for class ('not GNRA') and 1 for the other ('GNRA')
        Dense(1, activation='sigmoid')  # Sigmoid for binary question.
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),  # optimizer=RMSprop(lr=0.001),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)

    return model

In [7]:
def make_model2(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    model = Sequential([
        # The input shape is 14x4
        # This is the first convolution
        Conv1D(64, 3, strides=1, activation='relu', padding='same',
               input_shape=(14, 4),
               kernel_initializer='he_normal',
               bias_initializer='zeros'),
        MaxPooling1D(pool_size=2, strides=2),
        Dropout(0.2),
        Conv1D(32, 3, strides=1, activation='relu', padding='same',
               kernel_initializer='he_normal',
               bias_initializer='zeros'),
        MaxPooling1D(pool_size=2, strides=2),
        Dropout(0.2),
        Flatten(),
        #  neuron hidden layer
        Dense(224, activation='relu', bias_initializer=output_bias),  # 14/2(maxpooling)=7, 32*7 = 224
        Dropout(0.2),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for class ('not GNRA') and 1 for the other ('GNRA')
        Dense(1, activation='sigmoid')  # Sigmoid for binary question.
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),  # optimizer=RMSprop(lr=0.001),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)

    return model

In [8]:
# retrain with class weights
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.54
Weight for class 1: 7.09


In [10]:
weighted_model1 = make_model1()
#weighted_model.load_weights(initial_weights)

weighted_history1 = weighted_model1.fit(
    x_train,
    y_train,
    validation_data = (x_test, y_test),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,

    # The class weights go here
    class_weight={0: 0.54, 1: 7.09})

Epoch 1/20


2024-04-29 13:53:30.729601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


3297/3297 [==============================] - 17s 4ms/step - loss: 0.2645 - tp: 3350.0000 - fp: 5803.0000 - tn: 43214.0000 - fn: 372.0000 - accuracy: 0.8829 - precision: 0.3660 - recall: 0.9001 - auc: 0.9575 - prc: 0.7135 - val_loss: 0.1168 - val_tp: 1807.0000 - val_fp: 1074.0000 - val_tn: 23440.0000 - val_fn: 49.0000 - val_accuracy: 0.9574 - val_precision: 0.6272 - val_recall: 0.9736 - val_auc: 0.9929 - val_prc: 0.9438
Epoch 2/20
3297/3297 [==============================] - 13s 4ms/step - loss: 0.1618 - tp: 3505.0000 - fp: 3011.0000 - tn: 46006.0000 - fn: 217.0000 - accuracy: 0.9388 - precision: 0.5379 - recall: 0.9417 - auc: 0.9831 - prc: 0.8566 - val_loss: 0.1015 - val_tp: 1820.0000 - val_fp: 914.0000 - val_tn: 23600.0000 - val_fn: 36.0000 - val_accuracy: 0.9640 - val_precision: 0.6657 - val_recall: 0.9806 - val_auc: 0.9951 - val_prc: 0.9577
Epoch 3/20
3297/3297 [==============================] - 13s 4ms/step - loss: 0.1313 - tp: 3558.0000 - fp: 2335.0000 - tn: 46682.0000 - fn: 164.0

In [11]:
weighted_model2 = make_model2()
#weighted_model.load_weights(initial_weights)

weighted_history2 = weighted_model2.fit(
    x_train,
    y_train, 
    validation_data = (x_test, y_test),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,

    # The class weights go here
    class_weight={0: 0.54, 1: 7.09})

Epoch 1/20
3297/3297 [==============================] - 15s 4ms/step - loss: 0.1995 - tp: 5303.0000 - fp: 4468.0000 - tn: 69063.0000 - fn: 275.0000 - accuracy: 0.9400 - precision: 0.5427 - recall: 0.9507 - auc: 0.9859 - prc: 0.8824 - val_loss: 0.1710 - val_tp: 1839.0000 - val_fp: 1524.0000 - val_tn: 22990.0000 - val_fn: 17.0000 - val_accuracy: 0.9416 - val_precision: 0.5468 - val_recall: 0.9908 - val_auc: 0.9951 - val_prc: 0.9335
Epoch 2/20
3297/3297 [==============================] - 13s 4ms/step - loss: 0.1144 - tp: 3582.0000 - fp: 2056.0000 - tn: 46961.0000 - fn: 140.0000 - accuracy: 0.9584 - precision: 0.6353 - recall: 0.9624 - auc: 0.9906 - prc: 0.9035 - val_loss: 0.0735 - val_tp: 1827.0000 - val_fp: 609.0000 - val_tn: 23905.0000 - val_fn: 29.0000 - val_accuracy: 0.9758 - val_precision: 0.7500 - val_recall: 0.9844 - val_auc: 0.9976 - val_prc: 0.9740
Epoch 3/20
3297/3297 [==============================] - 13s 4ms/step - loss: 0.0934 - tp: 3616.0000 - fp: 1523.0000 - tn: 47494.0000 

In [12]:
plot_metrics(weighted_history1, weighted_history2)

In [13]:
train_predictions_weighted1 = weighted_model1.predict(x_train, batch_size=BATCH_SIZE)
test_predictions_weighted1 = weighted_model1.predict(x_test, batch_size=BATCH_SIZE)

1649/1649 [==============================] - 2s 955us/step


In [14]:
train_predictions_weighted2 = weighted_model2.predict(x_train, batch_size=BATCH_SIZE)
test_predictions_weighted2 = weighted_model2.predict(x_test, batch_size=BATCH_SIZE)

1649/1649 [==============================] - 2s 939us/step


In [15]:
# Results of test data set. 
weighted_results1 = weighted_model1.evaluate(x_test, y_test,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model1.metrics_names, weighted_results1):
  print(name, ': ', value)
print()

plot_cm(y_test, test_predictions_weighted1)

loss :  0.06392601132392883
tp :  1841.0
fp :  461.0
tn :  24053.0
fn :  15.0
accuracy :  0.9819492101669312
precision :  0.7997393608093262
recall :  0.9919180870056152
auc :  0.9980196952819824
prc :  0.969272792339325

Non-GNRA Detected (True Negatives):  24053
Non-GNRA Incorrectly Detected (False Positives):  461
GNRA Missed (False Negatives):  15
GNRA Detected (True Positives):  1841
Total GNRA:  1856


In [16]:
# Results of test data set. 
weighted_results2 = weighted_model2.evaluate(x_test, y_test,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model2.metrics_names, weighted_results1):
  print(name, ': ', value)
print()

plot_cm(y_test, test_predictions_weighted2)

loss :  0.06392601132392883
tp :  1841.0
fp :  461.0
tn :  24053.0
fn :  15.0
accuracy :  0.9819492101669312
precision :  0.7997393608093262
recall :  0.9919180870056152
auc :  0.9980196952819824
prc :  0.969272792339325

Non-GNRA Detected (True Negatives):  24236
Non-GNRA Incorrectly Detected (False Positives):  278
GNRA Missed (False Negatives):  18
GNRA Detected (True Positives):  1838
Total GNRA:  1856


In [17]:
plot_roc("Conf1_Train Weighted", y_train, train_predictions_weighted1, color=colors[0])
plot_roc("Conf1_Test Weighted", y_test, test_predictions_weighted1, color=colors[0], linestyle='--')
plot_roc("Conf2_Train Weighted", y_train, train_predictions_weighted2, color=colors[1])
plot_roc("Conf2_Test Weighted", y_test, test_predictions_weighted2, color=colors[1], linestyle='--')


plt.legend(loc='lower right');

In [18]:
plot_prc("Conf1_Train Weighted", y_train, train_predictions_weighted1, color=colors[0])
plot_prc("Conf1_Test Weighted", y_test, test_predictions_weighted1, color=colors[0], linestyle='--')
plot_prc("Conf2_Train Weighted", y_train, train_predictions_weighted2, color=colors[1])
plot_prc("Conf2_Test Weighted", y_test, test_predictions_weighted2, color=colors[1], linestyle='--')
# plt.xlim([0, 1.2])
plt.ylim([0, 1.2])

plt.legend(loc='upper right');